In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier

from scipy import stats
from imblearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

test_id = test['id']

In [3]:
X = train.drop(['loan_status'], axis=1)
y = train['loan_status']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

vars_cat = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
vars_num = X_train.select_dtypes(include=['float64', 'int64']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, vars_num),
    ('cat', categorical_transformer, vars_cat),
])

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

In [4]:
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

param_dist = {'learning_rate': stats.uniform(0.01, 0.99),
              'max_depth': stats.randint(2, 12),
              'subsample': stats.uniform(0.1, 1),
              'colsample_bytree': stats.uniform(0.1, 1),
              'n_estimators': stats.randint(50, 1000),
              'reg_lambda': stats.uniform(0.0001, 1),
              'reg_alpha': stats.uniform(0.0001, 1)
              }

classifier = LGBMClassifier(use_label_encoder=False,random_state=42)
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
lgbm = RandomizedSearchCV(classifier,param_dist,cv=cv, verbose=1,scoring='roc_auc',n_iter=10,random_state=42)

lgbm.fit(X_train,y_train)
print(f'AUC CV: {round(lgbm.best_score_,2)}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Info] Number of positive: 4705, number of negative: 28135
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000931 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1153
[LightGBM] [Info] Number of data points in the train set: 32840, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.143270 -> initscore=-1.788389
[LightGBM] [Info] Start training from score -1.788389
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

In [5]:
lgbm = LGBMClassifier(
    **lgbm.best_params_, 
    random_state=42
)

# Train the model on the training set
lgbm.fit(X_train, y_train)

# Predictions (classes) and probabilities on the test set
y_pred = lgbm.predict(X_val)  
y_pred_prob = lgbm.predict_proba(X_val)[:, 1]  

# Model evaluation
auc_score = roc_auc_score(y_val, y_pred_prob)
print(f"AUC-ROC Score: {auc_score:.4f}")

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy Score: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

[LightGBM] [Info] Number of positive: 5882, number of negative: 35169
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1159
[LightGBM] [Info] Number of data points in the train set: 41051, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.143285 -> initscore=-1.788268
[LightGBM] [Info] Start training from score -1.788268
AUC-ROC Score: 0.9571
Accuracy Score: 0.9536

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     15126
           1       0.92      0.73      0.82      2468

    accuracy                           0.95     17594
   macro avg       0.94      0.86      0.89     17594
weighted avg       0.95      0.95      0.95     17594



In [9]:
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1_score = f1_score(y_val, y_pred, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1_score)

Accuracy: 0.9536205524610663
Precision: 0.952605108909391
Recall: 0.9536205524610663
F1 Score: 0.9513124736278887


In [6]:
test = preprocessor.transform(test)

y_pred_prob = lgbm.predict_proba(test)[:,1]
predictions = pd.DataFrame(y_pred_prob)


predictions = predictions.reset_index().rename(columns={
    'index':'id',
    0:'loan_status'
})

predictions['id'] = test_id


In [7]:
y_pred_prob

array([0.97672014, 0.02494523, 0.58068426, ..., 0.01312046, 0.32757343,
       0.9402675 ])

In [8]:
# predictions.to_csv("light_gbm_proba.csv", index=False)